In [23]:
mlist = [200311084,
 160311986,
 200311082,
 170311921,
 160311248,
 160311215,
 210311744,
 170311288,
 170311156,
 200311026]
kardexArr = []
for matricula in mlist:
    kardexArr.append(obtenerKardex(matricula))

In [2]:
pip install pygad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 KB 2.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import pygad as pg
import time
import requests

In [5]:
d16 = pd.read_csv('./datos_2016.csv')
d17 = pd.read_csv('./datos_2017.csv')
d18 = pd.read_csv('./datos_2018.csv')
d19 = pd.read_csv('./datos_2019.csv')
d2021 = pd.read_csv('./2020_2021.csv')

In [6]:
asignaturas = pd.read_csv('asignaturas2.csv',encoding = 'ISO-8859-1')

In [7]:
plan_2016 = asignaturas.query('plan == "IDP2016"')

In [8]:
oferta = pd.read_csv('oferta_ideio.csv',encoding = 'ISO-8859-1')

In [9]:
seriaciones = pd.read_csv('seriacion.csv')

In [10]:
cargas16_19 = pd.concat([d16,d17,d18,d19])
cargas20_21 = d2021

In [11]:
cargas16_19 = cargas16_19.query('carrera == "Ingeniería en Datos e Inteligencia Organizacional"')

In [12]:
cargas16_19 = cargas16_19[['Unnamed: 0', 'carrera', 'periodo', 'clave', 'asignatura',
       'promediofinal', 'docente', 'ceveval_global', 'ceneval_analitico',
       'ceneval_matematico', 'ceneval_lengua', 'ceneval_esp',
       'n_matricula']]

In [13]:
matriculas16 = cargas16_19.query('n_matricula > 160000000 and n_matricula < 170000000')['n_matricula'].unique()
matriculas17 = cargas16_19.query('n_matricula > 170000000 and n_matricula < 180000000')['n_matricula'].unique()
matriculas18 = cargas16_19.query('n_matricula > 180000000 and n_matricula < 190000000')['n_matricula'].unique()
matriculas19 = cargas16_19.query('n_matricula > 190000000 and n_matricula < 200000000')['n_matricula'].unique()
matriculas20 = cargas20_21.query('n_matricula > 200000000 and n_matricula < 210000000')['n_matricula'].unique()
matriculas21 = cargas20_21.query('n_matricula > 210000000 and n_matricula < 220000000')['n_matricula'].unique()

In [14]:
matriculas = np.concatenate((matriculas16, matriculas17,matriculas18,matriculas19,matriculas20,matriculas21))

In [15]:
cargas20_21 = cargas20_21.rename(columns={'programa': 'carrera'})

In [16]:
formato_cargas20_21 = cargas20_21[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]
formato_cargas16_19 = cargas16_19[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]

formato_cargas = pd.concat([formato_cargas16_19,formato_cargas20_21])

In [17]:
oferta = oferta[['Clave','Nombre', 'Maestro', 'Lunes','Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado']]

In [18]:
oferta.loc[36,'Miercoles'] = '13:00-15:00'

In [19]:
def formatoHora(hora):
    if len(hora) == 11:
        return hora
    elif len(hora) == 10:
        return '0' + hora
    elif len(hora) == 9:
        return '0' + hora[0:5] + '0' + hora[5:]
    elif len(hora) == 1:
        return '-'

In [20]:
oferta['Lunes'] = oferta['Lunes'].apply(lambda horas: formatoHora(horas))
oferta['Martes'] = oferta['Martes'].apply(lambda horas: formatoHora(horas))
oferta['Miercoles'] = oferta['Miercoles'].apply(lambda horas: formatoHora(horas))
oferta['Jueves'] = oferta['Jueves'].apply(lambda horas: formatoHora(horas))
oferta['Viernes'] = oferta['Viernes'].apply(lambda horas: formatoHora(horas))

In [21]:
def obtenerKardex(mat = -1):
    if(mat == -1):
        mat = np.random.choice(matriculas)
    return formato_cargas.query('n_matricula == ' + str(mat)).sort_values('periodo')
def obtenerCreditos(kardex):
    claves = kardex.query('promediofinal >= 7')['clave'].unique()
    totalCreditos = 0
    for clave in claves:
        if(clave[0:2] == 'LI' or clave[0:2] == 'TA' or clave[0:2] == 'AD'):
            continue
        totalCreditos += plan_2016.query('clave == "' + clave + '"')['creditos'].values[0]
    return totalCreditos
def materiaHaSidoAprobada(kardex,clave):
    if len(kardex.query('clave == "' + clave + '" and promediofinal >= 7')) == 0:
        return False
    else:
        return True
def respetaSeriacion(clave,kardex):
    if len(seriaciones.query('ser2 == "' + clave + '"')) == 0:
        return True
    else:
        #Se guarda en necesarias la lista de asignaturas necesarias para llevar la asignura "clave"
        necesarias = seriaciones.query('ser2 == "' + clave + '"')['ser1'].unique()
        
        #Se recorre la lista para comprobar si el estudiante ya pasó las asignaturas necesarias, en caso de que le falte 
        #al menos una, entonces no se respeta la seriación.
        for necesaria in necesarias:
            if not(materiaHaSidoAprobada(kardex,necesaria)):
                return False
        return True
#Filtra las materias que no cumplen las restricciones 1 y 2

def obtenerOfertaUtil(kardex,oferta):
    #Se eliminan las materias en la oferta que ya han sido aprobadas
    aprobadas = kardex.query('promediofinal >= 7')['clave'].unique()
    for clave in aprobadas:
        oferta = oferta.query('Clave != "' + clave + '"')
        
    #Se eliminan las materias que el alumno no puede llevar por la seriación
    ofertaUtilIndex = set(oferta.index.values)
    for idx in oferta.index:
        if not(respetaSeriacion(oferta.loc[idx]['Clave'],kardex)):
            ofertaUtilIndex.remove(idx)
            
    #Se eliminan prácticas profesionales y proyecto terminal
    for i in range(len(oferta['Clave'].values)):
        clave = oferta['Clave'].values[i]
        if clave[0:3] == 'PID' or clave == 'IT0427':
            try:
                ofertaUtilIndex.remove(oferta.index.values[i])
            except:
                continue
    ofertaUtil = oferta.loc[list(ofertaUtilIndex)]
    
    #Se eliminan materias de elección libre de primer y segundo ciclo
    if(materiaHaSidoAprobada(kardex,'IL0102')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0160"')
    if(materiaHaSidoAprobada(kardex,'ID0160')):
        ofertaUtil = ofertaUtil.query('Clave != "IL0102"')
    
    if(materiaHaSidoAprobada(kardex,'IT0103')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0161"')
    if(materiaHaSidoAprobada(kardex,'ID0161')):
        ofertaUtil = ofertaUtil.query('Clave != "IT0103"')
        
    if(materiaHaSidoAprobada(kardex,'ID0264')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0262"')
    if(materiaHaSidoAprobada(kardex,'ID0262')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0264"')
        
    if(materiaHaSidoAprobada(kardex,'ID0263')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0265"')
    if(materiaHaSidoAprobada(kardex,'ID0265')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0263"')
        
    
    return ofertaUtil
def generarSolucionAleatoria(ofertaUtil):
    solucion = [-1] * 9
    for i in range(9):
        gen = np.random.randint(-1,len(ofertaUtil))
        if gen == -1:
            solucion[i] = -1
        else:
            solucion[i] = ofertaUtil.index[gen]
    return solucion
def obtenerDatosCarga(solucion,ofertaUtil):
    solucionU = np.array(solucion)
    solucionU = np.unique(solucionU)
    solucionU = solucionU[solucionU >= 0]
    
    return ofertaUtil.loc[solucionU]
def comprobarTraslapacion(solucion,ofertaUtil):
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    
    for dia in dias:
        horarioDia = datosCarga.sort_values(dia)[dia].values
        for i in range(len(horarioDia)):
            if horarioDia[i] == '-':
                continue
            horaInicioI = int(horarioDia[i][0:2])
            horaFinI = int(horarioDia[i][6:8])
            
            for j in range(i+1,len(horarioDia)):
                horaInicioJ = int(horarioDia[j][0:2])
                horaFinJ = int(horarioDia[j][6:8])
                
                if not(horaFinJ  <= horaInicioI or horaInicioJ >= horaFinI):
                    return True
    return False
def esValido(solucion,ofertaUtil):
    #Si se repite una materia es inválido
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    if len(datosCarga['Clave'].unique()) < len(datosCarga):
        return False
    #Si se traslapan dos materias es inválido
    if comprobarTraslapacion(solucion,ofertaUtil):
        return False
    return True
def generarSolucionValida(ofertaUtil):
    bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    while not(esValido(bitsDeCarga,ofertaUtil)):
        bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    return bitsDeCarga
def obtenerHorario(carga,ofertaUtil):
    primeraHoraMinima = 24
    ultimaHoraMaxima = 0
    datosCarga = obtenerDatosCarga(carga,ofertaUtil)
    horario = pd.DataFrame({
        'Hora': ['7:00-8:00','8:00-9:00','9:00-10:00','10:00-11:00','11:00-12:00','12:00-13:00','13:00-14:00','14:00-15:00','15:00-16:00','16:00-17:00','17:00-18:00','18:00-19:00','19:00-20:00','20:00-21:00','21:00-22:00'],
        'Lunes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Martes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Miercoles': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Jueves': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Viernes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Sabado': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
    })
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])
            
            primeraHoraMinima = min(primeraHoraMinima,horaInicio)
            ultimaHoraMaxima = max(ultimaHoraMaxima,horaFin)
            
            nombre = datosCarga.iloc[i]['Nombre']

            for hora in range(horaInicio,horaFin):
                horario.loc[hora-7,dia]=nombre
    return horario[(primeraHoraMinima-7):(ultimaHoraMaxima-6)]
def obtenerMateriasReprobadas(kardex):
    materiasReprobadas = kardex.query('promediofinal < 7')['clave'].unique()
    materiasReprobadasFinal = []
    for clave in materiasReprobadas:
        aprobado = len(kardex.query('clave == "' + clave + '" and promediofinal >= 7'))
        if aprobado == 0:
            materiasReprobadasFinal.append(clave)
    return materiasReprobadasFinal
def UpMR(kardex,solucion,ofertaUtil):
    materiasReprobadas = obtenerMateriasReprobadas(kardex)
    if len(materiasReprobadas) == 0:
        return 0
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    utilidadTotal = 0
    
    for clave in materiasReprobadas:
        cargado = len(datosCarga.query('Clave == "' + clave + '"'))
        if cargado == 1:
            utilidadTotal += 1
            
    
    #Normalización
    utilidad = (utilidadTotal)/(len(materiasReprobadas))
    return utilidad
def UpCC(solucion,ofertaUtil):
    alpha = 3
    utilidad = [alpha**3,alpha**2,alpha,1]
    utilidadTotal = 0
    
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    
    claves = datosCarga['Clave'].unique()
    for i in range(len(claves)):
        if claves[i][0:2] == 'AD'or claves[i][0:2] == 'TA' or claves[i][0:2] == 'LI' or claves[i][0:2] == 'PI':
            continue
            
        ciclo = int(plan_2016.query('clave == "' + claves[i] + '"')['ciclos'])
        utilidadTotal += utilidad[ciclo-1]
    
    #Normalización
    utilidadNorm = (utilidadTotal)/(utilidad[0]*9)
    return utilidadNorm
def CpHL(solucion,ofertaUtil):
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    costoTotal = 0
    hlMax = 0
    
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    
    for dia in dias:
        datosCarga = datosCarga.sort_values(dia)
        if datosCarga[dia].iloc[len(datosCarga)-2] == '-':
            continue
        
        ultimaHoraFin = 0
        for i in range(len(datosCarga)):
            if datosCarga.iloc[i][dia] == '-':
                continue
            if ultimaHoraFin == 0:
                ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
                primeraHoraDia = int(datosCarga.iloc[i][dia][0:2])
                continue
            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            costoTotal += (horaInicio - ultimaHoraFin)
            ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
        hlMax += (ultimaHoraFin - primeraHoraDia - 2)
    
    if hlMax == 0:
        return 0
    #Normalización
    costo = (costoTotal)/(hlMax)
    return costo
# El estudiante trabaja por las mañanas tiempo completo
disp_est_1 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True]
})


# El estudiante trabaja por las tardes tiempo completo
disp_est_2 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Martes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Sabado": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False]
})

# El estudiante trabaja por las mañanas medio tiempo
disp_est_3 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Martes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Miercoles": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Jueves": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Viernes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Sabado": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False]
})
# El estudiante trabaja por las tardes medio tiempo
disp_est_4 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Martes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True]
})
disp_est_5 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
})
disponibilidadArr = [disp_est_1,disp_est_2,disp_est_3,disp_est_4,disp_est_5]
def CpDH(solucion,ofertaUtil,disponibilidad):
    disponibilidadTotal = sum(disponibilidad['Lunes']) + sum(disponibilidad['Martes']) + sum(disponibilidad['Miercoles']) + sum(disponibilidad['Jueves']) + sum(disponibilidad['Viernes']) + sum(disponibilidad['Sabado'])
    if disponibilidadTotal == 90:
        return 0
    
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    costoTotal = 0

    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])

            for hora in range(horaInicio,horaFin):
                if not(disponibilidad.query('hora == ' + str(hora))[dia].values[0]):
                    costoTotal += 1
                    
    #Normalización
    costo = (costoTotal)/(90 - disponibilidadTotal)
    return costo
def obtenerDesempeno(solucion,cromosomaIndice):
    if not(esValido(solucion,ofertaUtil)):
        return 0
    pesos = np.array([1,1,1,1])   
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    x = np.array([upcc,upmr,cpdh,cphl])
    x = x*pesos
    
    x[2] =  pesos[2]- x[2]
    x[3] = pesos[3] - x[3]
    
    return sum(x)
def evaluarSolucion(solucion):
    pesos = np.array([1,1,1,1])   
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    print("La carga evaluada tiene:")
    print("\t" + str(round(100*upcc,2)) + "% de utilidad por las materias de primeros ciclos llevadas.")
    print("\t" + str(round(100*upmr,2)) + "% del total de las materias reprobadas del estudiante.") 
    print("\t" + str(round(100*cpdh,2)) + "% de costo por sobreposición con las horas donde el estudiante no está disponible.")
    print("\t" + str(round(100*cphl,2)) + "% de costo por el total de horas libres por día.")

In [ ]:
num_generations = 100

for kardex in kardexArr:
    print('Iniciando proceso de matricula: ' + str(kardex.iloc[0]["n_matricula"]))
    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    #Cruce: Uniforme
    #Mutacion: Aleatoria
    instancia1 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia1.run()
    tiempoejecucion1 = time.time() - start_time
    generacionDeMejorSolucion1 = instancia1.best_solution_generation
    solution1, solution_fitness1, solution_idx = instancia1.best_solution()
    
    desempenos = instancia1.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero1 = sum(desempenos) / (len(desempenos))
    minimo1 = min(desempenos)
    
    print("--- Instancia 1 terminada en: %s segundos---" % (tiempoejecucion1))
    print("Desempeño: " + str(solution_fitness1))
    print("Solucion: " + str(solution1))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion1))
    print('Peor solucion: ' + str(minimo1))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero1))

    #Cruce: Un punto
    #Mutacion: Aleatoria
    instancia2 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))


    start_time = time.time()
    instancia2.run()
    tiempoejecucion2 = time.time() - start_time
    generacionDeMejorSolucion2 = instancia2.best_solution_generation
    solution2, solution_fitness2, solution_idx = instancia2.best_solution()
    
    desempenos = instancia2.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero2 = sum(desempenos) / (len(desempenos))
    minimo2 = min(desempenos)
    
    print("--- Instancia 2 terminada en: %s segundos---" % (tiempoejecucion2))
    print("Desempeño: " + str(solution_fitness2))
    print("Solucion: " + str(solution2))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion2))
    print('Peor solucion: ' + str(minimo2))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero2))
    
    #Cruce: Dos puntos
    #Mutacion: Aleatoria
    instancia3 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia3.run()
    tiempoejecucion3 = time.time() - start_time
    generacionDeMejorSolucion3 = instancia3.best_solution_generation
    solution3, solution_fitness3, solution_idx = instancia3.best_solution()
    
    desempenos = instancia3.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero3 = sum(desempenos) / (len(desempenos))
    minimo3 = min(desempenos)
    
    print("--- Instancia 3 terminada en: %s segundos---" % (tiempoejecucion3))
    print("Desempeño: " + str(solution_fitness3))
    print("Solucion: " + str(solution3))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion3))
    print('Peor solucion: ' + str(minimo3))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero3))

    #Cruce: Uniforme
    #Mutacion: Adaptativa
    instancia4 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia4.run()
    tiempoejecucion4 = time.time() - start_time
    generacionDeMejorSolucion4 = instancia4.best_solution_generation
    solution4, solution_fitness4, solution_idx = instancia4.best_solution()
    
    desempenos = instancia4.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero4 = sum(desempenos) / (len(desempenos))
    minimo4 = min(desempenos)
    
    print("--- Instancia 4 terminada en: %s segundos---" % (tiempoejecucion4))
    print("Desempeño: " + str(solution_fitness4))
    print("Solucion: " + str(solution4))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion4))
    print('Peor solucion: ' + str(minimo4))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero4))

    #Cruce: Un punto
    #Mutacion: Adaptativa
    instancia5 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia5.run()
    tiempoejecucion5 = time.time() - start_time
    generacionDeMejorSolucion5 = instancia5.best_solution_generation
    solution5, solution_fitness5, solution_idx = instancia5.best_solution()
    
    desempenos = instancia5.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero5 = sum(desempenos) / (len(desempenos))
    minimo5 = min(desempenos)
    
    print("--- Instancia 5 terminada en: %s segundos---" % (tiempoejecucion5))
    print("Desempeño: " + str(solution_fitness5))
    print("Solucion: " + str(solution5))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion5))
    print('Peor solucion: ' + str(minimo5))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero5))

    #Cruce: Dos puntos
    #Mutacion: Adaptativa
    instancia6 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia6.run()
    tiempoejecucion6 = time.time() - start_time
    generacionDeMejorSolucion6 = instancia6.best_solution_generation
    solution6, solution_fitness6, solution_idx = instancia6.best_solution()
    
    desempenos = instancia6.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero6 = sum(desempenos) / (len(desempenos))
    minimo6 = min(desempenos)
    
    print("--- Instancia 6 terminada en: %s segundos---" % (tiempoejecucion6))
    print("Desempeño: " + str(solution_fitness6))
    print("Solucion: " + str(solution6))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion6))
    print('Peor solucion: ' + str(minimo6))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero6))
    
    #Cruce: Uniforme
    #Mutacion: Aleatoria
    instancia7 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia7.run()
    tiempoejecucion7 = time.time() - start_time
    generacionDeMejorSolucion7 = instancia7.best_solution_generation
    solution7, solution_fitness7, solution_idx = instancia7.best_solution()
    
    desempenos = instancia7.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero7 = sum(desempenos) / (len(desempenos))
    minimo7 = min(desempenos)
    
    print("--- Instancia 1 terminada en: %s segundos---" % (tiempoejecucion7))
    print("Desempeño: " + str(solution_fitness1))
    print("Solucion: " + str(solution1))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion1))
    print('Peor solucion: ' + str(minimo1))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero1))

    #Cruce: Un punto
    #Mutacion: Aleatoria
    instancia2 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))


    start_time = time.time()
    instancia2.run()
    tiempoejecucion2 = time.time() - start_time
    generacionDeMejorSolucion2 = instancia2.best_solution_generation
    solution2, solution_fitness2, solution_idx = instancia2.best_solution()
    
    desempenos = instancia2.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero2 = sum(desempenos) / (len(desempenos))
    minimo2 = min(desempenos)
    
    print("--- Instancia 2 terminada en: %s segundos---" % (tiempoejecucion2))
    print("Desempeño: " + str(solution_fitness2))
    print("Solucion: " + str(solution2))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion2))
    print('Peor solucion: ' + str(minimo2))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero2))
    
    #Cruce: Dos puntos
    #Mutacion: Aleatoria
    instancia3 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia3.run()
    tiempoejecucion3 = time.time() - start_time
    generacionDeMejorSolucion3 = instancia3.best_solution_generation
    solution3, solution_fitness3, solution_idx = instancia3.best_solution()
    
    desempenos = instancia3.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero3 = sum(desempenos) / (len(desempenos))
    minimo3 = min(desempenos)
    
    print("--- Instancia 3 terminada en: %s segundos---" % (tiempoejecucion3))
    print("Desempeño: " + str(solution_fitness3))
    print("Solucion: " + str(solution3))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion3))
    print('Peor solucion: ' + str(minimo3))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero3))

    #Cruce: Uniforme
    #Mutacion: Adaptativa
    instancia4 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia4.run()
    tiempoejecucion4 = time.time() - start_time
    generacionDeMejorSolucion4 = instancia4.best_solution_generation
    solution4, solution_fitness4, solution_idx = instancia4.best_solution()
    
    desempenos = instancia4.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero4 = sum(desempenos) / (len(desempenos))
    minimo4 = min(desempenos)
    
    print("--- Instancia 4 terminada en: %s segundos---" % (tiempoejecucion4))
    print("Desempeño: " + str(solution_fitness4))
    print("Solucion: " + str(solution4))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion4))
    print('Peor solucion: ' + str(minimo4))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero4))

    #Cruce: Un punto
    #Mutacion: Adaptativa
    instancia5 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia5.run()
    tiempoejecucion5 = time.time() - start_time
    generacionDeMejorSolucion5 = instancia5.best_solution_generation
    solution5, solution_fitness5, solution_idx = instancia5.best_solution()
    
    desempenos = instancia5.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero5 = sum(desempenos) / (len(desempenos))
    minimo5 = min(desempenos)
    
    print("--- Instancia 5 terminada en: %s segundos---" % (tiempoejecucion5))
    print("Desempeño: " + str(solution_fitness5))
    print("Solucion: " + str(solution5))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion5))
    print('Peor solucion: ' + str(minimo5))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero5))

    #Cruce: Dos puntos
    #Mutacion: Adaptativa
    instancia6 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia6.run()
    tiempoejecucion6 = time.time() - start_time
    generacionDeMejorSolucion6 = instancia6.best_solution_generation
    solution6, solution_fitness6, solution_idx = instancia6.best_solution()
    
    desempenos = instancia6.last_generation_fitness.copy()
    desempenos = desempenos[desempenos != 0]
    if len(desempenos) > 0:
        desempenoPoblacionNoCero6 = sum(desempenos) / (len(desempenos))
    minimo6 = min(desempenos)
    
    print("--- Instancia 6 terminada en: %s segundos---" % (tiempoejecucion6))
    print("Desempeño: " + str(solution_fitness6))
    print("Solucion: " + str(solution6))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion6))
    print('Peor solucion: ' + str(minimo6))
    print('Promedio desempeño: ' + str(desempenoPoblacionNoCero6))
    
    requests.post('https://resultados-api.azurewebsites.net/registrar2', json = {
            'n_matricula': str(kardex.iloc[0]["n_matricula"]),
            'tiempoEjecucion1': str(tiempoejecucion1),'mejorGeneracion1': str(generacionDeMejorSolucion1),'solucion1': str(solution1),'desempeno1': str(solution_fitness1),'desempenoPoblacionNoCero1': str(desempenoPoblacionNoCero1),'desempenoMinimo1': str(minimo1),
            'tiempoEjecucion2': str(tiempoejecucion2),'mejorGeneracion2': str(generacionDeMejorSolucion2),'solucion2': str(solution2),'desempeno2': str(solution_fitness2),'desempenoPoblacionNoCero2': str(desempenoPoblacionNoCero2),'desempenoMinimo2': str(minimo2),
            'tiempoEjecucion3': str(tiempoejecucion3),'mejorGeneracion3': str(generacionDeMejorSolucion3),'solucion3': str(solution3),'desempeno3': str(solution_fitness3),'desempenoPoblacionNoCero3': str(desempenoPoblacionNoCero3),'desempenoMinimo3': str(minimo3),
            'tiempoEjecucion4': str(tiempoejecucion4),'mejorGeneracion4': str(generacionDeMejorSolucion4),'solucion4': str(solution4),'desempeno4': str(solution_fitness4),'desempenoPoblacionNoCero4': str(desempenoPoblacionNoCero4),'desempenoMinimo4': str(minimo4),
            'tiempoEjecucion5': str(tiempoejecucion5),'mejorGeneracion5': str(generacionDeMejorSolucion5),'solucion5': str(solution5),'desempeno5': str(solution_fitness5),'desempenoPoblacionNoCero5': str(desempenoPoblacionNoCero5),'desempenoMinimo5': str(minimo5),
            'tiempoEjecucion6': str(tiempoejecucion6),'mejorGeneracion6': str(generacionDeMejorSolucion6),'solucion6': str(solution6),'desempeno6': str(solution_fitness6),'desempenoPoblacionNoCero6': str(desempenoPoblacionNoCero6),'desempenoMinimo6': str(minimo6),
    })

    print("\n\nPROCESO TERMINADO EN " + str(tiempoejecucion6 + tiempoejecucion1 + tiempoejecucion2 + tiempoejecucion3 + tiempoejecucion4 + tiempoejecucion5))
    print('--------------------------------------------------------------------')

Iniciando proceso de matricula: 200311084
--- Instancia 1 terminada en: 96.13942956924438 segundos---
Desempeño: 2.0864197530864197
Solucion: [ 79  77  78 108  81  75  73  76  80]
Mejor generacion: 6
Peor solucion: 1.5864197530864197
Promedio desempeño: 1.874748685526955
--- Instancia 2 terminada en: 110.83742189407349 segundos---
Desempeño: 2.0488459473966723
Solucion: [108 103  87  75  81  35  56 116  79]
Mejor generacion: 9
Peor solucion: 1.5531039313545223
Promedio desempeño: 1.8551889676855136
--- Instancia 3 terminada en: 101.33253979682922 segundos---
Desempeño: 2.135802469135802
Solucion: [ 46  43  58  75  79  81  80  93 103]
Mejor generacion: 6
Peor solucion: 1.6246246246246248
Promedio desempeño: 1.923100631333
--- Instancia 4 terminada en: 229.84106349945068 segundos---
Desempeño: 2.1275720164609053
Solucion: [79 43 83 -1 57 73 43 80 75]
Mejor generacion: 5
Peor solucion: 1.3450911228689006
Promedio desempeño: 2.0222651037043278
--- Instancia 5 terminada en: 223.273512601852